In [55]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.model import deepar
# from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator
from gluonts.dataset import common
from gluonts.model import deepar
from gluonts.mx.distribution import DistributionOutput, GaussianOutput,NegativeBinomialOutput,StudentTOutput
from gluonts.evaluation import make_evaluation_predictions
from gluonts.mx.trainer.callback import TrainingHistory
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import datetime
from itertools import islice
from pathlib import Path
from sklearn import preprocessing
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scipy.stats
import datetime
from dateutil.relativedelta import relativedelta
import optuna
warnings.filterwarnings("ignore")

mx.random.seed(546781345)
np.random.seed(546781345)

In [56]:
df = pd.read_csv('monthly_data.csv')

In [57]:
df

,Unnamed: 0,datum,RemediMax,VitaFlux,SynaptraZol,NeuroRevive,Immunoxen,RemediSol,Vitalisol,RestoraZen
0,0,30-01-2017,127.69000,99.090000,152.100000,878.030000,354.000000,50.000000,112.000000,48.200000
1,1,27-02-2017,133.32000,126.050000,177.000000,1001.900000,347.000000,31.000000,122.000000,36.200000
2,2,30-03-2017,137.44000,92.950000,147.655000,779.275000,232.000000,20.000000,112.000000,85.400000
3,3,29-04-2017,113.10000,89.475000,130.900000,698.500000,209.000000,18.000000,97.000000,73.700000
4,4,30-05-2017,101.79000,119.933000,132.100000,628.780000,270.000000,23.000000,107.000000,123.700000
5,5,29-06-2017,112.07000,94.710000,122.900000,548.225000,323.000000,23.000000,57.000000,109.300000
6,6,30-07-2017,117.06000,95.010000,129.300000,491.900000,348.000000,21.000000,61.000000,69.100000
7,7,30-08-2017,134.79000,99.780000,123.800000,583.850000,420.000000,29.000000,37.000000,70.800000
8,8,29-09-2017,108.78000,109.094000,122.100000,887.820000,399.000000,14.000000,115.000000,58.800000
9,9,30-10-2017,154.75000,185.241000,191.600000,1856.815000,472.000000,30.000000,182.000000,74.500000


In [58]:
df['quarters'] = pd.to_datetime(df['datum']).dt.quarter

In [59]:
freq='M'
prediction_length=10

In [60]:
import optuna
from optuna.samplers import TPESampler

In [61]:
df_train = df[:-10]

In [62]:
df_predict = df

In [63]:
items = ['RemediMax', 'VitaFlux', 'SynaptraZol',
       'NeuroRevive', 'Immunoxen', 'RemediSol', 'Vitalisol', 'RestoraZen']

In [64]:
dynamic_real = pd.DataFrame(columns=items)

cat_data = pd.DataFrame(columns = ['ID','Therapeutic_area'])
cat_data['ID'] = items
cat_data['Therapeutic_area'] = ['Neuroscience','Immunology','Oncology',
                               'Diabetes','Anti-infectives','Immunology','Diabetes','Neuroscience']
label_encoder = preprocessing.LabelEncoder()
cat_data['Therapeutic_area'] = label_encoder.fit_transform(cat_data['Therapeutic_area'])
cat_data

,ID,Therapeutic_area
0,RemediMax,3
1,VitaFlux,2
2,SynaptraZol,4
3,NeuroRevive,1
4,Immunoxen,0
5,RemediSol,2
6,Vitalisol,1
7,RestoraZen,3


In [65]:
train_ds = ListDataset([
    {
        FieldName.TARGET: df_train[id].values,
        FieldName.START: df_train['datum'].values[0],
        FieldName.FEAT_STATIC_CAT:[cat_data[cat_data['ID']==id]['Therapeutic_area'].values[0]],
        FieldName.FEAT_DYNAMIC_REAL:[df_train['quarters'].values.T]
        
    }
    for id in items
],freq='M')

In [66]:
predict_ds = ListDataset([
    {
        FieldName.TARGET: df_predict[id].values,
        FieldName.START: df_train['datum'].values[0],
        FieldName.FEAT_STATIC_CAT:[cat_data[cat_data['ID']==id]['Therapeutic_area'].values[0]],
        FieldName.FEAT_DYNAMIC_REAL:[df_predict['quarters'].values.T]
    }
    for id in items
],freq='M')

In [67]:
def SMAPE(predictions,data):
    SMAPE = []
    data.reset_index(inplace=True)
    for id in items:
        pred_y = pd.DataFrame(predictions[id])
        pred_y.columns = ['pred_y']
        Output = pd.concat([data[id],pred_y],axis=1)
        Output['Diff_Val']=abs(Output[id]-Output['pred_y'])
        Output['sum_Val']=abs(Output[id]) + abs(Output['pred_y'])
        Output['Div_val']=Output['Diff_Val']/(Output['sum_Val']+0.0001)
        smape = (2*sum(Output['Div_val']))/Output.shape[0] 
        SMAPE.append(smape)
    return sum(SMAPE)/len(SMAPE)

In [70]:
def objective(trial):
    num_layers= trial.suggest_int('num_layers',2,4)  
    num_cells = trial.suggest_int('num_cells',32,64)
    learning_rate= trial.suggest_float('learning_rate',0.001,0.01)
    dropout_rate = trial.suggest_float('dropout_rate',0.01,0.1)
    history = TrainingHistory()
    estimator = deepar.DeepAREstimator(
                        freq=freq, 
                        prediction_length=prediction_length,
                        use_feat_static_cat=True,
                        cardinality = [cat_data['Therapeutic_area'].nunique()],
                        use_feat_dynamic_real=False,
                        num_layers=num_layers,
                        num_cells=num_cells,
                        dropout_rate = dropout_rate,
                        cell_type='lstm',
                        distr_output =NegativeBinomialOutput(),# Distribution to be used
                        trainer=Trainer(epochs=50,learning_rate=learning_rate,callbacks=[history]))
    predictor = estimator.train(training_data=train_ds)
    mx.random.seed(546781345)
    np.random.seed(546781345)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset=predict_ds,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)
    results_mean = pd.DataFrame()
    results_median = pd.DataFrame()
    actual = pd.DataFrame()
    results_mean['date'] = tss[0].index[-10:]
    results_median['date'] = tss[0].index[-10:]
    actual['date'] = tss[0].index[-10:]
    for i,id in zip(range(len(items)),items):
        results_mean[id] = forecasts[i].mean
        results_median[id] = forecasts[i].median
        actual[id] = tss[i].values.reshape(-1)[-10:]
    smape = SMAPE(results_median,actual)
    return smape
sampler = TPESampler(seed=546781345)
study = optuna.create_study(direction='minimize',sampler = sampler)
study.optimize(objective,n_trials=3)

[I 2023-06-28 02:33:54,832] A new study created in memory with name: no-name-2b87a720-1916-4649-8319-d95031f179ad
100%|██████████| 50/50 [00:02<00:00, 18.12it/s, epoch=50/50, avg_epoch_loss=-.891]
[I 2023-06-28 02:36:14,225] Trial 0 finished with value: 0.3053229686367558 and parameters: {'num_layers': 4, 'num_cells': 40, 'learning_rate': 0.006141287252343434, 'dropout_rate': 0.02875209047078995}. Best is trial 0 with value: 0.3053229686367558.
100%|██████████| 50/50 [00:02<00:00, 18.27it/s, epoch=50/50, avg_epoch_loss=-.984]
[I 2023-06-28 02:38:31,340] Trial 1 finished with value: 0.27835625783191065 and parameters: {'num_layers': 4, 'num_cells': 37, 'learning_rate': 0.009112679919026447, 'dropout_rate': 0.0718871964591016}. Best is trial 1 with value: 0.27835625783191065.
100%|██████████| 50/50 [00:02<00:00, 19.23it/s, epoch=50/50, avg_epoch_loss=-.185]
[I 2023-06-28 02:40:41,982] Trial 2 finished with value: 0.3264776841271669 and parameters: {'num_layers': 3, 'num_cells': 61, 'lear

In [71]:
train_ds = ListDataset([
    {
        FieldName.TARGET: df_predict[id].values,
        FieldName.START: df_train['datum'].values[0],
        FieldName.FEAT_STATIC_CAT:[cat_data[cat_data['ID']==id]['Therapeutic_area'].values[0]],       
    }
    for id in items
],freq='M')

In [72]:
history = TrainingHistory()
estimator = deepar.DeepAREstimator(
                    freq=freq, 
                    prediction_length=prediction_length,
                    use_feat_static_cat=True,
                    cardinality = [cat_data['Therapeutic_area'].nunique()],
                    use_feat_dynamic_real=False,
                    num_layers=study.best_params['num_layers'],
                    num_cells=study.best_params['num_cells'],
                    dropout_rate = study.best_params['dropout_rate'],
                    cell_type='lstm',
                    distr_output =NegativeBinomialOutput(),
                    trainer=Trainer(epochs=100,learning_rate=study.best_params['learning_rate'],callbacks=[history]))
predictor = estimator.train(training_data=train_ds)

100%|██████████| 50/50 [00:02<00:00, 18.50it/s, epoch=100/100, avg_epoch_loss=-.338]


In [73]:
valid = pd.read_csv('monthly_validate.csv')
valid.reset_index(inplace=True)

In [74]:
valid

,index,Unnamed: 0,datum,RemediMax,VitaFlux,SynaptraZol,NeuroRevive,Immunoxen,RemediSol,Vitalisol,RestoraZen
0,0,60,2022-01-30,179.70,222.351,99.70,1660.612,295.2,23.0,386.0,41.30
1,1,61,2022-02-27,133.73,142.155,110.20,1001.212,249.4,12.0,226.0,69.50
2,2,62,2022-03-30,154.52,113.118,83.35,941.050,301.4,19.0,257.0,169.50
3,3,63,2022-04-29,161.39,100.165,88.10,647.650,299.4,22.0,259.0,179.10
4,4,64,2022-05-30,168.04,97.258,104.10,703.562,265.8,26.0,322.0,135.40
5,5,65,2022-06-29,151.54,101.627,103.20,610.000,193.0,25.0,142.0,156.04
6,6,66,2022-07-30,181.00,103.541,92.80,649.800,250.6,20.0,115.0,105.20
7,7,67,2022-08-30,181.91,88.269,84.20,518.100,237.0,26.0,145.0,97.30
8,8,68,2022-09-29,161.07,111.437,93.50,984.480,227.8,16.0,161.0,109.10
9,9,69,2022-10-30,44.37,37.300,20.65,295.150,86.0,7.0,37.0,11.13


In [75]:
mx.random.seed(546781345)
np.random.seed(546781345)

forecast_it, ts_it = make_evaluation_predictions(
dataset=train_ds,  # test dataset
predictor=predictor,  # predictor
num_samples=100,  # number of sample paths we want for evaluation
)
forecasts = list(forecast_it)
tss = list(ts_it)

In [76]:
results_median = pd.DataFrame()
actual = pd.DataFrame()
results_mean = pd.DataFrame()

In [77]:
for i,id in zip(range(len(items)),items):
    results_median[id] = forecasts[i].median
    results_mean[id] = forecasts[i].mean

In [84]:
def smape(float(actual),float(forecast)):
    return np.mean(2*abs(actual-forecast)/(abs(actual)+abs(forecast)))

SyntaxError: invalid syntax (<ipython-input-84-cbdb1550480a>, line 1)

In [47]:
def mape(actual,forecast):
    return np.mean(abs(actual - forecast)/actual)

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*data
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return (m, m-h, m+h)

In [ ]:
predictions = pd.DataFrame()
pred_temp=[]
time = []
ID = []
for i,id in zip(range(len(items)),items):
    results_mean[id] = forecasts[i].mean
    results_median[id] = forecasts[i].median
    actual[id] = tss[i].values.reshape(-1)[-12:]
    quantile_1[id] = forecasts[i].quantile(0.05)
    quantile_2[id] = forecasts[i].quantile(0.95)
for id,pred in zip(df_train['ID'].unique(),prediction):
#     predictions[id] = pred.mean
    dat = pd.DataFrame(data=pred.samples)
    date =datetime.datetime.strptime('2020-08-01', "%Y-%m-%d").date()
    for i in range(dat.shape[1]):
        conf = mean_confidence_interval(dat.loc[:,i].values.flatten(),confidence = 95/100)
        pred_temp.append(conf)
        if freq=='M':
            date += relativedelta(months=+1)
        time.append(date)
        ID.append(id)
predictions = pd.DataFrame(pred_temp,columns = ['Mean','Lower_Bound','Upper_bound'])
#         predictions[predictions<0]=0
predictions = predictions.round(0)
predictions.insert(0,'ID',ID)
predictions.insert(1,'time',time)

In [ ]:
results_mean = pd.DataFrame()
results_median = pd.DataFrame()
actual = pd.DataFrame()
quantile_1 = pd.DataFrame()
quantile_2 = pd.DataFrame()
results_mean['date'] = tss[0].index[-12:]
results_median['date'] = tss[0].index[-12:]
actual['date'] = tss[0].index[-12:]
quantile_1['date'] = tss[0].index[-12:]
quantile_2['date'] = tss[0].index[-12:]
for i,id in zip(range(len(items)),items):
    results_mean[id] = forecasts[i].mean
    results_median[id] = forecasts[i].median
    actual[id] = tss[i].values.reshape(-1)[-12:]
    quantile_1[id] = forecasts[i].quantile(0.05)
    quantile_2[id] = forecasts[i].quantile(0.95)

In [ ]:
actual= pd.melt(actual,id_vars='date',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'],
               value_name='actual')

In [ ]:
results_mean= pd.melt(results_mean,id_vars='date',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'],
               value_name='forecast_mean')

In [ ]:
results_median= pd.melt(results_median,id_vars='date',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'],
               value_name='forecast_median')

In [ ]:
quantile_1= pd.melt(quantile_1,id_vars='date',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'],
               value_name='quantile_0.05')

In [ ]:
quantile_2= pd.melt(quantile_2,id_vars='date',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'],
               value_name='quantile_0.95')

In [ ]:
final_results = pd.merge(actual,results_mean,on=['date','variable'],how='left')

In [ ]:
final_results = pd.merge(final_results,quantile_1,on=['date','variable'],how='left')

In [ ]:
final_results = pd.merge(final_results,quantile_2,on=['date','variable'],how='left')

In [ ]:
final_results['date'] = final_results['date'].dt.date

In [ ]:
final_results['model'] = 'deepar'
final_results['granularity'] = 'M'